<a href="https://colab.research.google.com/github/keeprich/AI-Trading-Bot/blob/main/place_simple_order.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install oandapyV20

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.6/51.6 kB 1.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for oandapyV20: filename=oandapyV20-0.7.2-py3-none-any.whl size=69778 sha256=509cb5534049bb8ea7e0ab55c4505469c3b2d832fc93ba42907db74f50e00d3d
  Stored in directory: /root/.cache/pip/wheels/8e/69/ab/a6da84a85b9bf3b5a98ca15c6c52b7854f32b10c70fe0531a1
Successfully built oandapyV20


In [ ]:
from oandapyV20 import API
from oandapyV20.endpoints.orders import OrderCreate
from oandapyV20.endpoints.accounts import AccountInstruments
import oandapyV20

# Replace with your actual API credentials

API_KEY = "7881b59039beb34197d5c0d3d16f0a7a-fa4ffbfebacce78af23ec0227bef49bd"
ACCOUNT_ID = "101-004-30239848-002"
api = API(access_token=API_KEY)


def get_instrument_precision(symbol):
    """Fetch instrument details to get the price precision."""
    try:
        response = api.request(AccountInstruments(accountID=ACCOUNT_ID))
        for instrument in response['instruments']:
            if instrument['name'] == symbol:
                return int(instrument['displayPrecision'])
    except oandapyV20.exceptions.V20Error as e:
        print(f"Error fetching instrument details: {e}")
    return 5  # Default precision if unable to retrieve

def place_test_order(symbol, action, lot_size, tp, sl):
    # Fetch the required decimal precision for the instrument
    precision = get_instrument_precision(symbol)

    # Format TP and SL prices dynamically based on the precision
    tp = f"{tp:.{precision}f}"
    sl = f"{sl:.{precision}f}"

    # Order data with formatted TP and SL
    order_data = {
        "order": {
            "units": str(lot_size) if action == "buy" else str(-lot_size),
            "instrument": symbol,
            "timeInForce": "FOK",
            "type": "MARKET",
            "positionFill": "DEFAULT",
            "takeProfitOnFill": {
                "price": tp
            },
            "stopLossOnFill": {
                "price": sl
            }
        }
    }

    # Print the order details before placing it
    print("Placing order with the following details:")
    print(f"Instrument: {symbol}")
    print(f"Action: {'BUY' if action == 'buy' else 'SELL'}")
    print(f"Lot Size: {lot_size}")
    print(f"Take Profit (TP): {tp}")
    print(f"Stop Loss (SL): {sl}")
    print(f"Order Data: {order_data}")

    try:
        # Place the order
        response = api.request(OrderCreate(ACCOUNT_ID, data=order_data))
        print("Order placed successfully. Response from server:")
        print(response)
    except oandapyV20.exceptions.V20Error as e:
        print("Error placing order:", e)

# Test values
symbol = "EUR_USD"        # Instrument
action = "buy"            # Buy action
lot_size = 1000           # Number of units
current_price = 1.0772    # Example current price

# Calculate TP and SL based on a simple percentage
tp = current_price * 1.005  # 0.5% above current price
sl = current_price * 0.995  # 0.5% below current price

# Call the function to place a test order
place_test_order(symbol, action, lot_size, tp, sl)


Placing order with the following details:
Instrument: EUR_USD
Action: BUY
Lot Size: 1000
Take Profit (TP): 1.08259
Stop Loss (SL): 1.07181
Order Data: {'order': {'units': '1000', 'instrument': 'EUR_USD', 'timeInForce': 'FOK', 'type': 'MARKET', 'positionFill': 'DEFAULT', 'takeProfitOnFill': {'price': '1.08259'}, 'stopLossOnFill': {'price': '1.07181'}}}
Order placed successfully. Response from server:
{'orderCreateTransaction': {'id': '40', 'accountID': '101-004-30239848-002', 'userID': 30239848, 'batchID': '40', 'requestID': '61303828526500733', 'time': '2024-11-08T12:43:22.087181429Z', 'type': 'MARKET_ORDER', 'instrument': 'EUR_USD', 'units': '1000', 'timeInForce': 'FOK', 'positionFill': 'DEFAULT', 'takeProfitOnFill': {'price': '1.08259', 'timeInForce': 'GTC'}, 'stopLossOnFill': {'price': '1.07181', 'timeInForce': 'GTC', 'triggerMode': 'TOP_OF_BOOK'}, 'reason': 'CLIENT_ORDER'}, 'orderFillTransaction': {'id': '41', 'accountID': '101-004-30239848-002', 'userID': 30239848, 'batchID': '40'